In [124]:
import keys
import pandas as pd
import requests
from pprint import pprint
import json
import numpy as np

In [58]:
df = pd.read_csv('merged.csv')

In [73]:
df.head(5)

,id,name,close_date,stage,address,lat,lng
0,0061a00000GcQdL,525 Water St SW- CMC Mgmt,3/8/18,Contract Sent,525 Water St SW Washington United States Distr...,NaN,NaN
1,0061a00000Mi8dC,Bozzuto - NJ - Park + Garden,6/30/18,Invoice Sent,1450 Garden St Hoboken United States New Jerse...,NaN,NaN
2,0061a00000M4PAU,Greystar- The Cameron,11/17/17,Deal Signed,8710 Cameron Street Silver Spring United State...,NaN,NaN
3,0061a00000MgelJ,JLL- 2035 NW Front Ave,1/31/18,Proposal Sent,2035 Front Ave Portland United States Oregon 9...,NaN,NaN
4,0061a00000KOF8z,International Monetary Fund- Spring Conference,1/31/18,Closed Won,"700 19th St NW Washington, United States Distr...",NaN,NaN


In [80]:
#df2 = df[(df['lat'].isnull()==True) & (df['address'].isnull()==False)]

In [170]:
#(df['lat'].isnull()==True) & 
#key = 'xxx'
key = keys.google
counter = 0
#loop through all records with an address, but no lat/long
for i, row in df[(df['lat'].isnull()==True) & 
                 (df['address'].isnull()==False)
                 #& (df['id']=='0061a00000GcQdL') #for test
                ].iterrows():
    #get the lat and long from google maps api
    address = row.address.replace(' ','+')
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address='+address+'=&key='+key
    response = requests.get(url)
    try:
        data = json.loads(response.text)
        #print(data)
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        #add it to the dataframe
        df.loc[df['id'] == row.id, 'lat'] = lat
        df.loc[df['id'] == row.id, 'lng'] = lng
        #print(address,lat,lng)
    except:
        print ('request failure for row', i, data)
    counter +=1
    if counter%10==0:
        print(counter,address,lat,lng)
        #break

request failure for row 57 {'results': [], 'status': 'ZERO_RESULTS'}
request failure for row 166 {'results': [], 'status': 'ZERO_RESULTS'}
request failure for row 370 {'results': [], 'status': 'ZERO_RESULTS'}
request failure for row 403 {'results': [], 'status': 'ZERO_RESULTS'}
request failure for row 494 {'results': [], 'status': 'ZERO_RESULTS'}
request failure for row 594 {'results': [], 'status': 'ZERO_RESULTS'}
request failure for row 692 {'results': [], 'status': 'ZERO_RESULTS'}
10 6425+Penn+Ave.+Pittsburgh+United+States+Pennsylvania+15206 40.4568686 -79.9171461
20 215+S+16th+St.+Philadelphia+United+States+Pennsylvania+19102 39.9490859 -75.1672045
request failure for row 760 {'results': [], 'status': 'ZERO_RESULTS'}
30 1180+West+Peachtree+St.
Ste.+500+Atlanta+United+States+Georgia+30309 33.7869866 -84.3882734
request failure for row 774 {'error_message': 'You have exceeded your daily request quota for this API. We recommend registering for a key at the Google Developers Console: h

In [174]:
#df.head(3000)

In [171]:
#number of uncompleted records
df[(df['lat'].isnull()==True) & (df['address'].isnull()==False)].shape

(25, 7)

In [173]:
#export to csv
df.to_csv('merged.csv',index_label=False)